In [3]:
import pandas as pd
from linearmodels.iv import IV2SLS
import statsmodels.formula.api as smf

In [5]:
df = pd.read_csv('./heat_death_iv=ndvi_2009.csv')
df.head()

,county_code,NAME,avg_ndvi,Notes_x,County_x,Year,Year Code,Deaths,Population,Crude Rate,Notes_y,County_y,Heat Wave Days Based on Daily Maximum Temperature
0,1001.0,Autauga,0.491605,NaN,"Autauga County, AL",2009.0,2009.0,44.0,54135.0,81.3,NaN,"Autauga County, AL",36
1,1003.0,Baldwin,0.426574,NaN,"Baldwin County, AL",2009.0,2009.0,160.0,179406.0,89.2,NaN,"Baldwin County, AL",30
2,1005.0,Barbour,0.558331,NaN,"Barbour County, AL",2009.0,2009.0,14.0,27657.0,Unreliable,NaN,"Barbour County, AL",31
3,1007.0,Bibb,0.570957,NaN,"Bibb County, AL",2009.0,2009.0,23.0,22941.0,100.3,NaN,"Bibb County, AL",55
4,1009.0,Blount,0.739535,NaN,"Blount County, AL",2009.0,2009.0,69.0,57341.0,120.3,NaN,"Blount County, AL",39


In [17]:
# data cleaning
df['crude_rate'] = df['Crude Rate']
df['Deaths'] = df['Deaths'].fillna(0)
df['heat_wave_days'] = df['Heat Wave Days Based on Daily Maximum Temperature']
# print(df['crude_rate'].isnull().sum())
# df['crude_rate'] = df['crude_rate'].fillna(0)
# df['crude_rate'] = pd.to_numeric(df['crude_rate'], errors='coerce')

In [19]:
# IV_1 - C(Year) + C(county_code)
formula = 'Deaths ~ 1+ C(Year) + C(county_code) + [heat_wave_days ~ avg_land_cover]'
iv2sls = IV2SLS.from_formula(formula, df).fit()

ValueError: Error encountered while checking for nulls in `heat_wave_days`: `heat_wave_days` contains null values after evaluation.

In [23]:
print(iv2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 Deaths   R-squared:                      0.9965
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9945
No. Observations:                7578   F-statistic:                -7.222e+13
Date:                Wed, Dec 11 2024   P-value (F-stat)                1.0000
Time:                        14:44:28   Distribution:               chi2(2702)
Cov. Estimator:                robust                                         
                                                                              
                                     Parameter Estimates                                     
                           Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------------
Intercept                     46.517     6.4787     7.1799     0.0000      33.819     

In [25]:
# IV_2 - C(Year)
formula = 'crude_rate ~ 1+ C(Year) + [heat_wave_days ~ avg_land_cover]'
iv2sls = IV2SLS.from_formula(formula, df).fit()
print(iv2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 Deaths   R-squared:                     -501.30
Estimator:                    IV-2SLS   Adj. R-squared:                -501.50
No. Observations:                7578   F-statistic:                    2.0056
Date:                Wed, Dec 11 2024   P-value (F-stat)                0.5713
Time:                        14:48:41   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
Intercept            -1293.8     997.07    -1.2976     0.1944     -3248.0      660.39
C(Year)[T.2009.0]    -30

In [31]:
# calculate wald estimator
first_stage = smf.ols("heat_wave_days ~ C(Year) + C(county_code) + avg_land_cover", data = df).fit()
reduced_form = smf.ols("Deaths ~ C(Year) + C(county_code) + avg_land_cover", data = df).fit()
reduced_form.params["Deaths"] / first_stage.params["Deaths"]

KeyError: 'Deaths'

In [33]:
# 第一阶段回归：因变量为内生变量，工具变量为 avg_land_cover
first_stage = smf.ols("heat_wave_days ~ C(Year) + C(county_code) + avg_land_cover", data=df).fit()

# 简化形式回归：因变量为最终结果变量
reduced_form = smf.ols("Deaths ~ C(Year) + C(county_code) + avg_land_cover", data=df).fit()

# 提取工具变量 avg_land_cover 的系数
wald_estimator = reduced_form.params["avg_land_cover"] / first_stage.params["avg_land_cover"]

print("Wald Estimator:", wald_estimator)


Wald Estimator: -0.011778768504440188


In [35]:
# 第一阶段回归：因变量为内生变量，工具变量为 avg_land_cover
first_stage = smf.ols("heat_wave_days ~ C(Year) + avg_land_cover", data=df).fit()

# 简化形式回归：因变量为最终结果变量
reduced_form = smf.ols("Deaths ~ C(Year) + avg_land_cover", data=df).fit()

# 提取工具变量 avg_land_cover 的系数
wald_estimator = reduced_form.params["avg_land_cover"] / first_stage.params["avg_land_cover"]

print("Wald Estimator:", wald_estimator)


Wald Estimator: 285.01460335719185
